# Querying Stream Metadata with the Rune Labs API/SDK

This tutorial demonstrates the basic steps of querying and exploring stream metadata using the Rune Labs API/SDK. This metadata describes and parameterized each stream of data, and is used in later steps to retrieve the desired data.

This tutorial follows: [Exploring Patient Devices with the Rune Labs API/SDK](03_exploring_patient_devices.ipynb).

For detailed information:
* [Rune Labs API documentation](https://docs.runelabs.io/stream/v2/)
* [Rune Labs SDK documentation](https://runeq.readthedocs.io/en/latest/)
* [Rune Labs open source code respository](https://github.com/rune-labs/opensource) (which includes this notebook)

---

## Set Up

Initialize the Rune SDK with your platform credentials, as described previously: [Getting Started with the Rune Labs API/SDK](01_getting_started_with_Rune_SDK.ipynb)

In [1]:
# Initialize the SDK.
from runeq import initialize

initialize()

To confirm that you have successfully initialized the SDK in your current script or notebook, pull your own information using the `get_current_user()` function.

In [2]:
# Get the ID and name of the current user, based on API credentials.
from runeq.resources.user import get_current_user

my_user = get_current_user()
print(my_user)

User(id="user-b9c372f2b315a6c6cfd9b5ef7eba81e5ef7866d1,user", name="Gavin Philips")


In [3]:
# Confirm user's current active org.
print('Active Org:', my_user.active_org_name)

Active Org: Rune Demo


---

## Patients

[Exploring Organizations and Patients with the Rune Labs API/SDK](02_exploring_organizations_and_patients.ipynb) includes instructions to copy a `patient_id` from a patient in your org. Paste that `patient_id` into the cell below.

We could also explore stream metadata across all patients in the org, but for simplicity, we'll focus on one in this tutorial.

In [4]:
# Set ID of example patient from your org.
patient_id = 'e3dd146f74714135a11128e99f1557f0'

---

## Stream Metadata

Finding and accessing data using our API/SDK is a two-step process:
1. Query for data streams using a desired set of parameters
    * Result: A set of stream metadata, which describes a set of streams
2. Use the resulting stream IDs to pull the desired data for a chosen time frame
    * Result: Actual data in a chosen format

We'll cover the first step here: querying for stream metadata.

Use the [get_patient_stream_metadata()](https://runeq.readthedocs.io/en/latest/pages/resources.html#runeq.resources.stream_metadata.get_patient_stream_metadata) function to perform each query. This function requires a `patient_id` parameter, which specifies the desired patient. It also accepts a number of other optional parameters, which can be used to focus the query.

To explore we'll start with just the ID of the example patient chosen above. By querying with just the patient ID, we'll retrieve metadata for all available streams associated with this patient.

In [5]:
# Query for all streams available for the desired patient.
from runeq.resources.stream_metadata import get_patient_stream_metadata
import pandas as pd

all_metadata = get_patient_stream_metadata(patient_id=patient_id)
all_metadata_df = all_metadata.to_dataframe()
# Display just the first five rows.
all_metadata_df.head()

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,log_type,measurement,id,severity,axis,source_device,sleep_status,hk_aggregation,function,user_entered
0,1.677327e+09,ingest-rune-events.0,IPl2_ivn,e3dd146f74714135a11128e99f1557f0,"{'name': 'Event', 'description': 'Occurrences ...",1.655103e+09,1.655142e+09,"{'category': 'patient_report', 'log_type': 'st...",patient_report,start,watch_recording_manual,baaeebde2cadb367cd13d8b72f0ef86fdf5e55e8574551...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.677327e+09,ingest-rune-events.0,IPl2_ivn,e3dd146f74714135a11128e99f1557f0,"{'name': 'Event', 'description': 'Occurrences ...",1.655103e+09,1.655147e+09,"{'category': 'patient_report', 'log_type': 'st...",patient_report,stop,watch_recording_manual,b6aa500b90c87403eb64d8dbc30740b961732e698d3609...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.655410e+09,ingest-strive-applewatch-md.0,IPl2_ivn,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.655103e+09,1.655146e+09,"{'category': 'symptom', 'severity': 'strong', ...",symptom,NaN,tremor,9528368d5168855d1c6138cfcba736f440d2b591a94813...,strong,NaN,NaN,NaN,NaN,NaN,NaN
3,1.655410e+09,ingest-strive-applewatch-md.0,IPl2_ivn,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.655103e+09,1.655146e+09,"{'category': 'symptom', 'measurement': 'dyskin...",symptom,NaN,dyskinesia,5781e118d1663e1e6ff3770f48d631c1fab3e85cff4879...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.655410e+09,ingest-strive-applewatch-md.0,IPl2_ivn,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.655103e+09,1.655146e+09,"{'category': 'symptom', 'severity': 'unknown',...",symptom,NaN,tremor,c0e6006ddb01a4069cb48bbfce25b32d31ecef264da4f0...,unknown,NaN,NaN,NaN,NaN,NaN,NaN


The stream metadata produced this query is returned as a [StreamMetadataSet](https://runeq.readthedocs.io/en/latest/pages/resources.html#runeq.resources.stream_metadata.StreamMetadataSet) object, which has several handy methods that will be utilized moving forward. The first of which is [to_dataframe()](https://runeq.readthedocs.io/en/latest/pages/resources.html#runeq.resources.stream_metadata.StreamMetadataSet.to_dataframe), which was used above to make the stream set easier to display.

In [6]:
type(all_metadata)

runeq.resources.stream_metadata.StreamMetadataSet

In the resulting stream set, there are quite a few streams available (represented as rows above), and quite a few parameters for each (represented as columns). The [SDK documentation for this function](https://runeq.readthedocs.io/en/latest/pages/resources.html#runeq.resources.stream_metadata.get_patient_stream_metadata) lists the possible parameters for data streams:

* `patient_id` – Patient ID of the desired patient
    * **required**
* `device_id` – Device ID of the data source device
    * discussed in [Exploring Patient Devices with the Rune Labs API/SDK](03_exploring_patient_devices.ipynb)
* `stream_type_id` – Stream type ID
    * discussed in **TODO: link to stream types notebook**
* `algorithm` – A versioned label that describes the process that was used to derive the data stream
    * typically one per data source, e.g., Apple Health, Medtronic Percept DBS
* `category` – A broad categorization of the data type (e.g. neural, vitals, etc)
* `measurement` – A specific label for what is being measured (e.g. heart_rate, step_count, etc).
* `**parameters` – Key/value pairs that label the stream
    * only apply to certain streams, as specified in the [algorithm documentation](https://drive.google.com/drive/folders/1GgyMHiDnXhPXCNbEvXG-nbWqQjBBVkE5?usp=sharing)

Note that some of the columns in the above are populated mostly with `NaN`. These fall in the last category, and only applied to certain streams.

Let's explore some of these parameters further:

---

### Algorithm

Data is parsed and ingested into the Rune Labs platform by different algorithms. Each data source has its own algorithm, and each algorithm has a numbered release version. Documentation for each algorithm, including the spec of each associated stream and its parameters, is available [here](https://drive.google.com/drive/folders/1GgyMHiDnXhPXCNbEvXG-nbWqQjBBVkE5?usp=sharing). Note that each spreadsheet has a separate sheet for each numbered algorithm version.

Examine the algorithms that have been used to ingest data for this patient:

In [9]:
# Display all unique algorithm values.
print(all_metadata_df.algorithm.unique())

['ingest-rune-events.0' 'ingest-strive-applewatch-md.0'
 'ingest-strive-applewatch-motion.0' 'ingest-strive-healthkit.0'
 'ingest-strive-healthkit.1']


This patient has data streams from:
* `ingest-rune-events.0` - Patient-reported events (PROs) from the StrivePD mobile app ([docs](https://docs.google.com/spreadsheets/d/1NMMfJunvSjmHWSr0r8iQs85BSWzxvlOqE36iej3jNPY/edit?usp=sharing))
* `ingest-strive-applewatch-md.0` - Tremor and dyskinesia metrics from Apple's MM4PD algorithms (a.k.a. movement disorder kit) ([docs](https://docs.google.com/spreadsheets/d/1cYH9MTW7y7KCKo4-tMNQ7mylnhCvZSPWdFhFNT6sX0M/edit?usp=sharing))
* `ingest-strive-applewatch-motion.0` - Raw accelerometry and gyro data from the Apple Watch (not available in standard StrivePD usage) ([docs](https://docs.google.com/spreadsheets/d/1vlmbWtRt5hniXkmtrqWNM8WW6B-41C9Y6KYV1WY1rLA/edit?usp=sharing))
* `ingest-strive-healthkit.0` - Various metrics from Apple's Health ecosystem ([docs](https://docs.google.com/spreadsheets/d/1yRVtxJueAn2-0FC4gMCdPtJyeZtzdMh5MdW4_XlDiJo/edit?usp=sharing))
* `ingest-strive-healthkit.1` - Various metrics from Apple's Health ecosystem ([docs](https://docs.google.com/spreadsheets/d/1yRVtxJueAn2-0FC4gMCdPtJyeZtzdMh5MdW4_XlDiJo/edit?usp=sharing)) **(new version)**

Use the `algorithm` parameter to query for data from a desired source, such as Apple Health:

In [10]:
# Query for all streams available for the Apple Health algorithm.
health_metadata = get_patient_stream_metadata(patient_id=patient_id, algorithm='ingest-strive-healthkit.1')
health_metadata_df = health_metadata.to_dataframe()

# Display available metrics.
print(health_metadata_df.measurement.unique())

['stand_time' 'heart_rate' 'heart_rate_motion_context' 'sleep_state'
 'step_count' 'walking_step_length' 'height' 'walking_running_distance'
 'exercise_time' 'heart_rate_variability' 'walking_steadiness'
 'walking_double_support_percentage' 'walking_asymmetry'
 'oxygen_saturation_percentage' 'barometric_pressure' 'heart_rate_resting'
 'walking_speed' 'stair_descent_speed' 'stair_ascent_speed' 'sleep_stage'
 'device_placement' 'utc_offset']


Note that some of these metrics fall into the "vitals" category, and their streams were also retrieved previously by querying for that category.

**It is recommended to always specify an algorithm in queries, to avoid unexpected results if a new algorithm version is released.**

For example, queries that do not specify the `ingest-strive-healthkit.1` algorithm may return additional streams from the `ingest-strive-healthkit.0` algorithm that are similar, but with an outdated set of parameters.

---

### Category

Each data stream has been tagged with a broad category to ease exploration.

Examine the categories of data available for this patient:

In [7]:
# Display all unique category values.
print(all_metadata_df.category.unique())

['patient_report' 'symptom' 'motion' 'vitals' 'device_info' 'sleep'
 'environment']


These broad categories may include various metrics from multiple sources. 

Use the `category` parameter to query for data of a desired category, such as vitals:

In [8]:
# Query for all streams available in the vitals category.
vitals_metadata = get_patient_stream_metadata(patient_id=patient_id, category='vitals')
vitals_metadata_df = vitals_metadata.to_dataframe()

# Display available metrics.
print(vitals_metadata_df.measurement.unique())

['heart_rate' 'heart_rate_motion_context' 'height'
 'heart_rate_variability' 'oxygen_saturation_percentage'
 'heart_rate_resting']


---

### Measurement

Each data stream has a measurement parameter, which describes the particular metric being quantified.

Examine the different measurements available for this patient:

In [11]:
# Display all unique measurement values.
print(all_metadata_df.measurement.unique())

['watch_recording_manual' 'tremor' 'dyskinesia' 'user' 'gravity' nan
 'stand_time' 'heart_rate' 'heart_rate_motion_context' 'diagnostics'
 'sleep_state' 'step_count' 'height' 'walking_running_distance'
 'walking_step_length' 'exercise_time' 'heart_rate_variability'
 'oxygen_saturation_percentage' 'walking_double_support_percentage'
 'walking_asymmetry' 'walking_steadiness' 'barometric_pressure'
 'heart_rate_resting' 'walking_speed' 'stair_ascent_speed'
 'stair_descent_speed' 'sleep_stage' 'device_placement' 'utc_offset'
 'mood']


These measurements describe data streams at a more granular level than the `category` and `algorithm` parameters.

Use the `measurement` parameter to query for data of a specific metric, such as tremor.

In [12]:
# Query for all streams available that measure tremor.
tremor_metadata = get_patient_stream_metadata(
    patient_id=patient_id, 
    measurement='tremor', 
)
tremor_metadata_df = tremor_metadata.to_dataframe()

tremor_metadata_df

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,severity,measurement,id
0,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'mild', 'm...",symptom,mild,tremor,f103d67147b77b4b66c348f23d4fd3291626e0a007571c...
1,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'strong', ...",symptom,strong,tremor,38b173174fb0f9f6c99cb6418742ebc9d7067a24cf6eeb...
2,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'all', 'me...",symptom,all,tremor,83257a46c877b96f4c639d2d3961401baf27d50b69f777...
3,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'slight', ...",symptom,slight,tremor,b4f10c508a1cdd9eabfc8370acdf7a85572db190b4cd0a...
4,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'moderate'...",symptom,moderate,tremor,8b38e92ec02396381886802591a7ff611aebe9cb5db67a...
5,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'none', 'm...",symptom,none,tremor,71249a55f7ce7b7654188d816746138bd2184eb5f4ce2f...
6,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'unknown',...",symptom,unknown,tremor,d41c647931013dc582c7ae2fc1f819368f293de7f2800b...
7,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Percentage', 'description': 'Percent...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'all', 'me...",symptom,all,tremor,99936f5a603339029f25ce78e118ea15822c71acf1668f...
8,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Percentage', 'description': 'Percent...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'strong', ...",symptom,strong,tremor,e5b276d4dc03be7ce183d30997951e32426c7948b14d14...
9,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Percentage', 'description': 'Percent...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'moderate'...",symptom,moderate,tremor,5090090a415ad3dbf9fac9e624aa234e7287a39f486678...


From one device (an Apple Watch), this patient has 14 streams that quantify tremor. They are all from the `ingest-strive-applewatch-md.0` algorithm, and match the spec given in [that algorithms documentation](https://docs.google.com/spreadsheets/d/1cYH9MTW7y7KCKo4-tMNQ7mylnhCvZSPWdFhFNT6sX0M/edit?usp=sharing). They also all fall into the `symptom` category.

There are two sets of seven streams each, with either the "Duration" or "Percentage" stream type (which are slightly different representations of the data). Each stream is separated by a `severity` parameter, which is not available for most data streams, but is applicable to measurements of tremor. The `severity` values include:
* `none` - no resting tremor detected
* `slight` - displacement less than 0.1 cm
* `mild` - displacement between 0.1 cm and 0.6 cm
* `moderate` - displacement between 0.6 cm and 2.2 cm
* `strong` - displacement greater than 2.2 cm
* `all` - resting tremor of any displacement detected (slight, mild, moderate, and strong combined)
* `unknown` - resting tremor undetermined (may be produced during times of active movement)

---

## Filtering Stream Metadata by Parameters

To narrow down the set of streams to just the desired data, use the [filter()](https://runeq.readthedocs.io/en/latest/pages/resources.html#runeq.resources.stream_metadata.StreamMetadataSet.filter) method of the `StreamMetadataSet` object.

In [13]:
# Query for symptom streams.
symptom_metadata = get_patient_stream_metadata(
    patient_id=patient_id,
    algorithm='ingest-strive-applewatch-md.0', 
    category='symptom', 
    stream_type_id='duration'
)
symptom_metadata.to_dataframe()

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,severity,measurement,id
0,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'mild', 'm...",symptom,mild,tremor,f103d67147b77b4b66c348f23d4fd3291626e0a007571c...
1,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'strong', ...",symptom,strong,tremor,38b173174fb0f9f6c99cb6418742ebc9d7067a24cf6eeb...
2,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'all', 'me...",symptom,all,tremor,83257a46c877b96f4c639d2d3961401baf27d50b69f777...
3,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'slight', ...",symptom,slight,tremor,b4f10c508a1cdd9eabfc8370acdf7a85572db190b4cd0a...
4,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'moderate'...",symptom,moderate,tremor,8b38e92ec02396381886802591a7ff611aebe9cb5db67a...
5,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'none', 'm...",symptom,none,tremor,71249a55f7ce7b7654188d816746138bd2184eb5f4ce2f...
6,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'unknown',...",symptom,unknown,tremor,d41c647931013dc582c7ae2fc1f819368f293de7f2800b...
7,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'measurement': 'dyskin...",symptom,NaN,dyskinesia,d40cf7d1979590c0caaa70722add44ab62ec2bad9624f4...


Filter the set of symptom streams to get just the dyskinesia streams:

In [14]:
# Filter for dyskinesia streams.
dyskinesia_metadata = symptom_metadata.filter(measurement='dyskinesia')
dyskinesia_metadata.to_dataframe()

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,measurement,id
0,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'measurement': 'dyskin...",symptom,dyskinesia,d40cf7d1979590c0caaa70722add44ab62ec2bad9624f4...


You can also filter with multiple parameters. For example, filter with `measurement` and `severity` to get just the tremor stream that includes all displacements.

In [15]:
# Filter for "tremor all" streams.
tremor_all_metadata = symptom_metadata.filter(measurement='tremor', severity='all')
tremor_all_metadata.to_dataframe()

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,severity,measurement,id
0,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'all', 'me...",symptom,all,tremor,83257a46c877b96f4c639d2d3961401baf27d50b69f777...


---

## Combining Parameters for Specific Queries

If you already know the exact set of desired parameters, you can query for them directly, and skip filtering.

Any number of parameters can be combined in a query to retrieve metadata for specific streams.

Use multiple parameters to find streams for this patient that quantify tremor of any displacement, from the desired device and current ingestion algorithm, and the "duration" representation:

In [16]:
# Query for the specific desired streams.
specific_metadata = get_patient_stream_metadata(
    patient_id=patient_id,
    algorithm='ingest-strive-applewatch-md.0', 
    measurement='tremor', 
    severity='all', 
    stream_type_id='duration'
)
specific_metadata_df = specific_metadata.to_dataframe()

specific_metadata_df

,created_at,algorithm,device_id,patient_id,stream_type,min_time,max_time,parameters,category,severity,measurement,id
0,1.668829e+09,ingest-strive-applewatch-md.0,LARICRKw,e3dd146f74714135a11128e99f1557f0,"{'name': 'Duration', 'description': 'Durations...",1.668225e+09,1.693247e+09,"{'category': 'symptom', 'severity': 'all', 'me...",symptom,all,tremor,83257a46c877b96f4c639d2d3961401baf27d50b69f777...


We've narrowed down our query to retrieve the metadata of exactly one stream.

We will use this stream metadata to pull the actual stream data in the next tutorial: [Pulling Stream Data](05_pulling_stream_data.ipynb).